In [1]:
import os

os.chdir("..")

In [ ]:
from datasetsforecast.m3 import M3
from utilsforecast.plotting import plot_series
import pandas as pd
from models.NBeatsStackMoe import NBeatsStackMoe
from models.NBeatsMoeLags import NBeatsMoeLags
from neuralforecast import NeuralForecast
from utils import load_dataset, train_test_split
from models.callbacks.probs_collector import GateValuesCollectorCallback
from neuralforecast.losses.numpy import smape
from neuralforecast.models import NBEATS
from neuralforecast.tsdataset import TimeSeriesDataset
import matplotlib.pyplot as plt
from datasets.load_data.gluonts_dataset import GluontsDataset
from models.hyper import config
import numpy as np

# Load the M3 dataset
# m3 = M3.load(directory='C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3\\', group='Monthly')[0]
m1, horizon, n_lags, freq_str, freq_int = GluontsDataset.load_everything("m1_monthly")

m1['y'] = m1['y'].astype(float)

# m4['ds'] = pd.to_datetime(m4['ds']).astype(int)

In [ ]:
horizon = horizon
input_size = n_lags

In [2]:
from neuralforecast.losses.numpy import smape
def calculate_smape(Y_test_df, Y_hat_df, forecast_col):
    """Calculate the sMAPE."""
    print(Y_test_df,Y_hat_df)

    y_true = Y_test_df['y'].values
    try:
        y_hat = Y_hat_df[forecast_col].values
    except Exception as exc:
        y_hat = Y_hat_df
   
    n_series = Y_test_df['unique_id'].nunique()
    try:
        y_true = y_true.reshape(n_series, -1)
        y_hat = y_hat.reshape(n_series, -1)
    except Exception as e:
        raise ValueError("Error reshaping arrays") from e


    return smape(y_true, y_hat)

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-17 21:36:10,480	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-17 21:36:10,643	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from functools import partial
from utilsforecast.losses import mase, rmse, mse, mae
from modelradar.evaluate.radar import ModelRadar
from matplotlib.cm import get_cmap

# Enable pgf backend for LaTeX-friendly output
matplotlib.use("pgf")

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False,
})

# Centralized model style settings
model_styles = {
    'N-BEATS': {
        'color': '#d62728',  # Muted Blue
        'linestyle': '-'
    },
    'N-BEATS-MOE': {
        'color': '#ff7f0e',  # Orange
        'linestyle': '-'
    },
    'MoEBlock': {
        'color': '#2ca02c',  # Green
        'linestyle': '-'
    },
    'SeasonalNaive': {
        'color': '#1f77b4',  # Red
        'linestyle': '-'
    },
}



def get_aspect_forecast_samples(radar, err_df, y_test, y_hats, models_dict, n_samples=4):
    """
    Get unique sample IDs for different aspects and their corresponding forecasts.
    Ensures no repeated unique_ids across aspects.
    """
    aspect_samples = {}
    used_uids = set()

    def get_unique_uids(candidates, label):
        new_uids = [uid for uid in candidates if uid not in used_uids]
        selected = list(np.random.choice(new_uids, min(n_samples, len(new_uids)), replace=False))
        used_uids.update(selected)
        aspect_samples[label] = selected

    # Hard
    hard_uids = radar.uid_accuracy.get_hard_uids(err_df, return_df=True).index.tolist()
    get_unique_uids(hard_uids, 'Hard')

    # Anomalies
    if 'anomaly_status' in y_test.columns:
        anomaly_uids = y_test[y_test['anomaly_status'] == 'Anomalies']['unique_id'].unique()
        get_unique_uids(anomaly_uids, 'Anomalies')

    # Seasonal
    if 'seas_str' in y_test.columns:
        seasonal_uids = y_test[y_test['seas_str'] == 'Seasonal']['unique_id'].unique()
        get_unique_uids(seasonal_uids, 'Seasonal')

    # Stationary
    if 'trend_str' in y_test.columns:
        stationary_uids = y_test[y_test['trend_str'] == 'Stationary']['unique_id'].unique()
        get_unique_uids(stationary_uids, 'Stationary')

    ## sample n_sample that are not any of the previous
    # "Other": sample n_samples unique_ids not already used in any aspect
    all_uids = set(y_test['unique_id'].unique())
    remaining_uids = [uid for uid in all_uids if uid not in used_uids]
    selected = remaining_uids[:n_samples]
    used_uids.update(selected)
    aspect_samples['Others'] = selected

    return aspect_samples

def plot_aspect_forecasts(aspect_samples, y_test, y_hats, models_dict, dataset_name):
    """
    Create 4x4 grid of forecasts by aspect
    """
    aspects = list(aspect_samples.keys())
    n_aspects = len(aspects)
    n_samples = 4
    
    fig, axes = plt.subplots(n_aspects, n_samples, figsize=(16, 12), sharex=True)
    if n_aspects == 1:
        axes = axes.reshape(1, -1)
    
    unique_ids = y_test['unique_id'].unique().tolist()
    

    # Get a colormap with as many distinct colors as models
    cmap = get_cmap("tab10")  # Can also try 'Set2', 'Dark2', 'Paired'
    colors = [cmap(i) for i in range(len(models_dict))]

    model_names = list(models_dict.keys())
    
    for aspect_idx, aspect in enumerate(aspects):
        sample_ids = aspect_samples[aspect]
        
        for sample_idx in range(n_samples):
            ax = axes[aspect_idx, sample_idx]
            
            if sample_idx < len(sample_ids):
                uid = sample_ids[sample_idx]
                
                # Get true values
                m_id = y_test[y_test['unique_id'] == uid]
                if len(m_id) == 0:
                    continue
                    
                m_idex = unique_ids.index(uid)
                
                # Plot true values
                ax.plot(m_id['y'].values, label='True', linewidth=2, color='black')
                
                # Plot forecasts for each model
                smape_texts = []

                for model_idx, (model_name, y_hat_col) in enumerate(models_dict.items()):
                    if model_idx < len(colors):
                        model_df = y_test[y_test["unique_id"] == uid].copy()
                        forecast = model_df[y_hat_col].values
                        true_vals = model_df['y'].values

                        # Plot forecast
                        style = model_styles.get(model_name, {'color': f'C{model_idx}', 'linestyle': '-'})
                        ax.plot(forecast, label=model_name,
                                color=style['color'],
                                linestyle=style['linestyle'],
                                linewidth=1.5)

                        # Calculate sMAPE and store text
                        smape_val = calculate_smape(model_df, model_df, y_hat_col)
                        print(smape_val)
                        smape_texts.append(f'{model_name}: {smape_val:.2f}')
                                    
                # Formatting
                if aspect_idx == 0 and sample_idx == 0:
                    ax.legend(fontsize=8, loc='upper right')
                
                if sample_idx == 0:
                    ax.set_ylabel(f'{aspect}', fontsize=12, fontweight='bold')
                
                if aspect_idx == n_aspects - 1:
                    ax.set_xlabel(r'$\tau \in \{t+1,..., t+H\}$', fontsize=12)
                
                title = f'{uid}' # \nsMAPE: ' + '\n'.join(smape_texts)
                ax.set_title(title, fontsize=12)
                ax.grid(True, alpha=0.3)
            else:
                # Hide empty subplots
                ax.axis('off')
    
    #plt.suptitle(f'Aspect-based Forecasts - {dataset_name.upper()}', 
    #            fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    
    return fig

def main():
    # Your existing setup code
    results_list = {
        'm1m': 'results,gluonts,m1_monthly.csv',
        'm1q': 'results,gluonts,m1_quarterly.csv',
        'm1y': 'results,gluonts,m1_yearly.csv',
        'tm': 'results,gluonts,tourism_monthly.csv',
        'tq': 'results,gluonts,tourism_quarterly.csv',
        'ty': 'results,gluonts,tourism_yearly.csv',
        'm3m': 'results,m3,Monthly.csv',
        'm3q': 'results,m3,Quarterly.csv',
        'm3y': 'results,m3,Yearly.csv',
        'm4m': 'results,m4,Monthly.csv',
        'm4q': 'results,m4,Quarterly.csv',
        'm4y': 'results,m4,Yearly.csv',
    }
    
    mase_func = partial(mase, seasonality=1)
    
    # Process each dataset
    for dataset, file_path in results_list.items():
        print(f"Processing {dataset}...")
        
        # Load results and setup radar
        df = pd.read_csv(file_path)
        train_path = re.sub('results', 'train', file_path)
        train = pd.read_csv(train_path)
        
        radar = ModelRadar(
            cv_df=df,
            metrics=[mase_func],
            model_names=['NBeatsMoe', 'NBEATS', 'NBeatsStackMoe', 'SeasonalNaive'],
            hardness_reference='SeasonalNaive',
            ratios_reference='NBEATS',
            cvar_quantile=0.75,
            hardness_quantile=0.75,
            agg_func='mean',
            train_df=train,
            rope=10
        )
        
        err = radar.evaluate(keep_uids=True)
        
        # Load your model predictions
        # This part needs to be adjusted based on how your y_hats are stored
        models_dict = {
            'N-BEATS-MOE': 'NBeatsStackMoe',
            'N-BEATS': 'NBEATS',     
            'MoEBlock': 'NBeatsMoe',
            'SeasonalNaive': 'SeasonalNaive',
        }
        
        # For demonstration, I'll create dummy data structure
        # Replace this with your actual data loading
        # print(f"You need to load y_test and y_hats for {dataset}")
        # print(f"Expected y_test path: {y_test_path}")
        
        # Uncomment and modify the following when you have the data:
        aspect_samples = get_aspect_forecast_samples(radar, err, df, df, models_dict)
        fig = plot_aspect_forecasts(aspect_samples, df, df, models_dict, dataset)
        plt.savefig(f'./pgfs/aspect_forecasts_{dataset}.pgf', bbox_inches='tight')
        plt.close()


main()

Processing m1m...


C:\Users\ricar\AppData\Local\Temp\ipykernel_20684\2339698202.py:102: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.


     unique_id          ds           y   NBeatsMoe      NBEATS  \
2960     ID431  1978-08-31  125.800003  126.803345  123.987206   
2961     ID431  1978-09-30  126.599998  126.824490  124.730680   
2962     ID431  1978-10-31  126.900002  128.391430  125.892044   
2963     ID431  1978-11-30  126.800003  127.680750  130.668910   
2964     ID431  1978-12-31  127.300003  128.510330  131.823430   
2965     ID431  1979-01-31  127.599998  130.576950  131.002600   
2966     ID431  1979-02-28  127.900002  130.676770  129.949700   
2967     ID431  1979-03-31  130.300003  129.290130  132.517910   

      NBeatsStackMoe  SeasonalNaive  is_anomaly anomaly_status  \
2960       125.02625     108.099998           0  Non-anomalies   
2961       126.06278     109.400002           0  Non-anomalies   
2962       127.58576     114.400002           0  Non-anomalies   
2963       128.06442     116.900002           0  Non-anomalies   
2964       129.58919     118.199997           0  Non-anomalies   
2965     

C:\Users\ricar\AppData\Local\Temp\ipykernel_20684\2339698202.py:167: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown


Processing m1q...
    unique_id          ds      y  NBeatsMoe     NBEATS  NBeatsStackMoe  \
308      ID55  1986-06-30  536.0  920.08460  906.06226       907.45940   
309      ID55  1986-09-30  769.0  965.57684  963.93240       961.60767   

     SeasonalNaive  is_anomaly anomaly_status       trend_str  seas_str  
308          806.0           1      Anomalies  Non-stationary  Seasonal  
309          892.0           0  Non-anomalies  Non-stationary  Seasonal       unique_id          ds      y  NBeatsMoe     NBEATS  NBeatsStackMoe  \
308      ID55  1986-06-30  536.0  920.08460  906.06226       907.45940   
309      ID55  1986-09-30  769.0  965.57684  963.93240       961.60767   

     SeasonalNaive  is_anomaly anomaly_status       trend_str  seas_str  
308          806.0           1      Anomalies  Non-stationary  Seasonal  
309          892.0           0  Non-anomalies  Non-stationary  Seasonal  
0.3686345143005536
    unique_id          ds      y  NBeatsMoe     NBEATS  NBeatsStackMoe  \

...

C:\Users\ricar\AppData\Local\Temp\ipykernel_20684\2339698202.py:167: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown


Processing m4y...


C:\Users\ricar\AppData\Local\Temp\ipykernel_20684\2339698202.py:102: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.


      unique_id  ds            y  NBeatsMoe     NBEATS  NBeatsStackMoe  \
99780     Y4265  51  4531.759516  4426.9380  4449.3040       4430.5520   
99781     Y4265  52  4898.680640  4543.0596  4639.3394       4565.8184   
99782     Y4265  53  4918.364163  4648.4766  4807.1367       4701.3267   
99783     Y4265  54  4964.132363  4774.0380  4920.7580       4885.6290   
99784     Y4265  55  5067.787935  4898.1170  5067.5444       4995.3374   
99785     Y4265  56  4770.389681  5034.8680  5187.1094       5138.1587   

       SeasonalNaive  is_anomaly anomaly_status       trend_str  seas_str  
99780    4311.517647           0  Non-anomalies  Non-stationary  Seasonal  
99781    4311.517647           1      Anomalies  Non-stationary  Seasonal  
99782    4311.517647           0  Non-anomalies  Non-stationary  Seasonal  
99783    4311.517647           0  Non-anomalies  Non-stationary  Seasonal  
99784    4311.517647           0  Non-anomalies  Non-stationary  Seasonal  
99785    4311.517647     

C:\Users\ricar\AppData\Local\Temp\ipykernel_20684\2339698202.py:167: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
